# Test autoFRK

**Title**: Test autoFRK Functionality

**Author**: Hsu, Yao-Chih

**Reviewer**: Xie, Yi-Xuan

**Version**: 1141020

**Description**: This script tests the autoFRK python version in different scenarios.

**Reference**: Resolution Adaptive Fixed Rank Kringing by ShengLi Tzeng & Hsin-Cheng Huang

## Install our python autoFRK

In [67]:
import sys
import numpy as np
import torch
print("=" * 50)
print("Python Environment Info")
print("=" * 50)
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version.split()[0]}")
print()

print("=" * 50)
print("Package Locations")
print("=" * 50)
print(f"Torch location: {torch.__file__}")
print(f"NumPy location: {np.__file__}")
print()

print("=" * 50)
print("PyTorch Info")
print("=" * 50)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA build: {torch.version.cuda}")
print()

print("=" * 50)
print("Test PyTorch Computation")
print("=" * 50)
x = torch.rand(3, 3)
print(f"Random tensor:\n{x}")
print(f"Sum: {x.sum().item():.4f}")

In [68]:
import shutil
if shutil.which("dot") is None:
    error_msg = "Graphviz 'dot' executable not found. Please install Graphviz from https://graphviz.org/download/ and ensure it is added to your system PATH. Then restart your computer to apply the changes."
    raise EnvironmentError(error_msg)

In [69]:
# install autoFRK in development mode
import os
import sys
module_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

!"{sys.executable}" -m pip uninstall -y autoFRK
!"{sys.executable}" -m pip install --upgrade pip build setuptools wheel matplotlib pandas torchviz graphviz
!"{sys.executable}" -m build {module_root}
!"{sys.executable}" -m pip install -e "{module_root}"

## Import modules

In [70]:
# import modules
import os
import sys
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from autoFRK import AutoFRK
from autoFRK.utils.utils import to_tensor, p

## Version

In [71]:
!pip show autoFRK

Name: autoFRK
Version: 1.1.2
Summary: autoFRK: Automatic Fixed Rank Kriging. The Python version with PyTorch
Home-page: https://github.com/Josh-test-lab/autoFRK-python
Author: ShengLi Tzeng, Hsin-Cheng Huang, Wen-Ting Wang, Yao-Chih Hsu
Author-email: 
License-Expression: GPL-3.0-or-later
Location: D:\Github\autoFRK-python\test\.venv-gpu\Lib\site-packages
Editable project location: D:\Github\autoFRK-python
Requires: colorlog, numpy, pandas, scipy, torch
Required-by: 


In [72]:
from importlib.metadata import metadata

meta = metadata("autoFRK")
for key in meta:
    print(f"{key}: {meta[key]}")

Metadata-Version: 2.4
Name: autoFRK
Version: 1.1.2
Summary: autoFRK: Automatic Fixed Rank Kriging. The Python version with PyTorch
Author: ShengLi Tzeng, Hsin-Cheng Huang, Wen-Ting Wang, Yao-Chih Hsu
Maintainer: Yao-Chih Hsu
License-Expression: GPL-3.0-or-later
Project-URL: Homepage, https://github.com/Josh-test-lab/autoFRK-python
Project-URL: Homepage, https://github.com/Josh-test-lab/autoFRK-python
Keywords: kriging,spatial statistics,torch,autoFRK,geostatistics
Classifier: Development Status :: 5 - Production/Stable
Classifier: Development Status :: 5 - Production/Stable
Classifier: Development Status :: 5 - Production/Stable
Classifier: Development Status :: 5 - Production/Stable
Classifier: Development Status :: 5 - Production/Stable
Classifier: Development Status :: 5 - Production/Stable
Classifier: Development Status :: 5 - Production/Stable
Requires-Python: >=3.10
Description-Content-Type: text/markdown
License-File: LICENSE
Requires-Dist: numpy>=1.23
Requires-Dist: numpy>=1.23

## Load data

In [73]:
# load data
datasets_path = f'test datasets/matrixForTest'
data = pd.read_csv(os.path.join(datasets_path, 'matrixForTest_data.csv'))
locs = pd.read_csv(os.path.join(datasets_path, 'matrixForTest_locs.csv'))
data_missing = pd.read_csv(os.path.join(datasets_path, 'matrixForTest_data_missing.csv'))

In [74]:
# data
data_missing

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,-0.652166,-5.366158,3.143780,NaN,-7.997662,1.136705,0.960326,3.201232,1.992290,2.001078,...,-0.448844,-0.577043,1.269652,0.347800,NaN,2.661689,-0.436828,-3.561380,-2.566957,3.640283
1,NaN,NaN,2.749028,0.729822,-4.332897,-2.829327,NaN,-2.724882,3.281578,-0.628886,...,-3.152476,-0.342614,7.929967,-2.177937,0.556117,-2.043759,4.111130,-5.352733,1.529268,-1.917891
2,-4.549512,-7.763640,-1.872094,2.033659,-5.619806,-2.764594,6.608074,-1.398568,0.612283,-2.144521,...,-11.434087,3.868390,NaN,7.577723,-6.269609,-1.125877,-13.487372,-0.227024,NaN,12.927128
3,-5.870963,-5.904798,-0.655968,7.860149,2.625713,-3.847751,7.439885,2.407433,3.273375,2.692631,...,-7.528925,6.847024,7.220447,4.332599,-3.998274,2.552754,NaN,3.684432,0.137623,7.370367
4,NaN,-8.789871,0.440326,6.932291,-0.442761,-4.145054,5.732851,0.655010,0.159350,-3.153300,...,NaN,4.540188,1.572350,7.350779,-3.499281,-0.169003,-10.041218,1.220481,-3.726887,8.765699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2.790659,0.130805,5.131183,NaN,-4.404798,NaN,1.197340,-0.427687,1.653997,4.622978,...,-0.952464,4.506945,-0.397343,-4.158340,0.220648,-1.642949,10.328322,-4.605859,1.894557,NaN
96,1.685645,1.215962,NaN,-3.899476,-1.595553,1.209884,-0.674625,-3.748975,-2.862852,1.977467,...,3.481229,0.753463,-4.839027,-5.032350,3.904153,6.617573,6.643477,3.399834,NaN,NaN
97,-1.395409,-3.042111,1.776225,-0.018072,-8.091349,-0.487496,-5.728769,0.954178,NaN,4.115169,...,-0.294438,1.971506,0.779557,-3.610582,NaN,4.502825,2.828657,-1.532787,5.843775,3.382403
98,-4.765434,-4.096297,-6.376321,3.124180,10.963501,-6.188876,NaN,4.612062,-2.310631,-7.859488,...,-4.794723,1.663460,3.688412,12.393074,NaN,-0.586811,-14.112674,6.708079,-2.698385,7.515730


In [75]:
locs

,Var1,Var2
0,0.310345,0.551724
1,0.517241,0.827586
2,0.413793,0.103448
3,0.517241,0.344828
4,0.689655,0.034483
...,...,...
95,0.275862,0.655172
96,0.344828,0.862069
97,0.379310,0.758621
98,0.620690,0.241379


## Convert data to tensor

In [79]:
# convert to tensor
data = to_tensor(data.to_numpy())
locs = to_tensor(locs.to_numpy())
data_missing = to_tensor(data_missing.to_numpy())

AttributeError: 'Tensor' object has no attribute 'to_numpy'

## Test on known locations

### Fit and predict

In [85]:

model = AutoFRK(
    logger_level=20
)
model.forward(
    data=data,
    loc=locs,
    requires_grad=True
)

pred = model.predict(
    obj = model.obj
)
pred

2025-10-23 18:23:10 - autoFRK.utils.logger - WARNING: Parameter "device" was not set. Value "cuda:0" detected and used.
2025-10-23 18:23:10 - autoFRK.utils.logger - INFO: Gradient tracking has been enabled for autoFRK.
2025-10-23 18:23:10 - autoFRK.utils.logger - INFO: Calculate TPS with rectangular coordinates.


{'M': tensor([[ 7.1395e+00,  5.1027e+00, -4.6409e+00,  ..., -1.6017e-01,
          -7.9489e-03, -5.3018e-02],
         [ 5.1027e+00,  3.7313e+00, -3.3570e+00,  ..., -1.0528e-01,
          -6.6983e-03, -2.2702e-02],
         [-4.6409e+00, -3.3570e+00,  3.1845e+00,  ...,  7.9533e-02,
           2.5039e-02,  3.0511e-02],
         ...,
         [-1.6017e-01, -1.0528e-01,  7.9533e-02,  ...,  4.7364e-02,
          -8.2130e-03, -1.1569e-03],
         [-7.9489e-03, -6.6983e-03,  2.5039e-02,  ..., -8.2130e-03,
           4.1754e-02,  1.9213e-03],
         [-5.3018e-02, -2.2702e-02,  3.0511e-02,  ..., -1.1569e-03,
           1.9213e-03,  4.8314e-02]], device='cuda:0', dtype=torch.float64,
        grad_fn=<MmBackward0>),
 's': tensor(0.2810, device='cuda:0', dtype=torch.float64, grad_fn=<SubBackward0>),
 'negloglik': tensor(8485.9507, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>),
 'w': tensor([[-1.5136, -1.8858, -0.2411,  ...,  0.1136, -0.6498,  3.4300],
         [-0.7637, -1.653

### Compute MSE

In [88]:
import torch.nn.functional as F

F.mse_loss(pred['pred.value'].cpu(), data.cpu()).item()

11.853958742968917

```r
> options(digits = 15)
> print(mse)
[1] 11.8539587159961 # R mse result
```

In [47]:
for i in range(data.shape[1]):
    y_pred = pred['pred.value'][:, i].cpu()
    y_true = data[:, i].cpu()

    tmp = F.mse_loss(y_pred, y_true)
    print(tmp.item())

4.899819535424482
4.356997964644021
4.5968031250585515
6.584888998387436
5.0627783350604565
5.198387900405552
5.379178460521051
5.218628562688229
5.398990525779933
4.8945461175293055
5.3917725343365985
4.8798856680470495
3.459437928814986
5.226067155511864
5.779119703281396
5.1690674387378195
5.584187167623827
5.828497802606243
5.1451699302224405
5.818733144610434
5.913050417984646
4.023205666845455
4.67978573382279
5.722322758929874
3.4086275775403743
4.57294428310656
4.73024022027814
5.613315750287496
5.528935481597575
5.390605581294012


### Compare with R result

```r
> temp # r residual result
 [1]  7.9766306122139898 12.0116817669945561  8.0081251964859330  3.5373058520311229 13.8528751726772583
 [6]  1.7858863206045086  9.8859536589469261 30.0424029115677840 27.0729660181976612  0.2159766252862333
[11]  0.0752655442993194  5.8887993596240475 12.5987205947255649 11.4932438517303979 37.7533887912337889
[16]  1.7491903038108714  4.2473651696074111  6.6476433759056297  4.3364578111025098  5.5149082237930296
[21] 23.4936795233438822  5.3612826806645080 10.3639904061309522 22.0650623239508086  3.2049127751009370
[26]  5.1869105672897371 30.8554320276935812  1.3647053212213505  9.0990348107428840 39.9289638829070839
```

經過確認，R 與 Python 的結果是大致一致的。
全時間點與各時間點的 MSE ，與 R 的結果皆相差不大，精準度介於小數點後6位元至10位元。

### Plotting functions

In [46]:
# for i in range(data.shape[1]):
#     y_pred = pred['pred.value'][:, i].detach().cpu()
#     y_true = data[:, i].detach().cpu()

#     tmp = F.mse_loss(y_pred, y_true)
#     print(tmp)

#     plt.figure(figsize=(8,5))
#     plt.plot(y_true, label='True Value', marker='o')
#     plt.plot(y_pred, label='Predicted Value', marker='x')
#     plt.xlabel('Sample Index')
#     plt.ylabel('Value')
#     plt.title(f'Prediction vs True Value at {i} Step with error {round(tmp.item(), 2)}')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

### Check gradient tracking

In [43]:
# from torchviz import make_dot
# SAVE_DIR = "gradient tracking/all known locations"
# os.makedirs(SAVE_DIR, exist_ok=True)

# for k, v in model.obj.items():
#     if isinstance(v, torch.Tensor):
#         print(f"{k}: requires_grad={v.requires_grad};  grad_fn={v.grad_fn}; is_leaf={v.is_leaf}")
#         file_path = os.path.join(SAVE_DIR, k)
#         dot = make_dot(v, show_attrs=False, show_saved=False)
#         dot.attr(dpi='100')
#         dot.render(file_path, format='pdf')

# for k, v in model.obj['G'].items():
#     if isinstance(v, torch.Tensor):
#         print(f"{k}: requires_grad={v.requires_grad};  grad_fn={v.grad_fn}; is_leaf={v.is_leaf}")
#         file_path = os.path.join(SAVE_DIR, k)
#         dot = make_dot(v, show_attrs=False, show_saved=False)
#         dot.attr(dpi='100')
#         dot.render(file_path, format='pdf')

# for k, v in pred.items():
#     if isinstance(v, torch.Tensor):
#         print(f"{k}: requires_grad={v.requires_grad};  grad_fn={v.grad_fn}; is_leaf={v.is_leaf}")
#         file_path = os.path.join(SAVE_DIR, k)
#         dot = make_dot(v, show_attrs=False, show_saved=False)
#         dot.attr(dpi='100')
#         dot.render(file_path, format='pdf')

## Test on unknown locations
### Train:Test = 7:3

In [48]:
## split data into train and test
train_data = data[:70, :]
test_data = data[70:, :]
train_locs = locs[:70, :]
test_locs = locs[70:, :]

### Fit and predict

In [90]:
## training on train data
model = AutoFRK()
model.forward(
    data=train_data,
    loc=train_locs,
    requires_grad=True
)

## predict on test data
pred = model.predict(
    newloc = test_locs
)

2025-10-23 19:56:50 - autoFRK.utils.logger - WARNING: Parameter "device" was not set. Value "cuda:0" detected and used.
2025-10-23 19:56:50 - autoFRK.utils.logger - INFO: Gradient tracking has been enabled for autoFRK.
2025-10-23 19:56:50 - autoFRK.utils.logger - INFO: Calculate TPS with rectangular coordinates.


18.88861956598081

### Compute MSE

In [ ]:
# evaluate mse on test data
F.mse_loss(pred['pred.value'].cpu(), test_data.cpu()).item()

In [50]:

for i in range(test_data.shape[1]):
    y_pred = pred['pred.value'][:, i].cpu()
    y_true = test_data[:, i].cpu()

    tmp = F.mse_loss(y_pred, y_true)
    print(tmp.item())

11.439409123174496
25.1077785246337
27.45007335660805
11.824834333929697
14.635020086195986
9.478019600169299
13.727797186522015
12.17243612405233
17.848309462343472
11.414195386451238
16.19007357020559
12.385319820230809
10.433204084473632
41.77850758190167
38.01213778618045
27.143946247068804
9.501057920916212
18.938319398553315
17.356615362871466
19.450701337805995
15.18002896748467
17.513520317484907
15.105158906121497
35.53883367201953
21.326198346843498
9.02735323123472
30.130724439918023
19.64006772628649
12.267852191409588
24.64109288633307


### Compare with R result

**R MSPE(ALL TIME): 18.8886195905131**

**PYTHON MSPE(ALL TIME): 18.88861955720493**

| Time | R MSPE | Python MSPE | Difference |
|------|--------|-------------|------------|
| 1 | 11.4394091240148 | 11.439409120905758 | 0.000000003108942 |
| 2 | 25.1077785518079 | 25.107778518456907 | 0.000000033350993 |
| 3 | 27.4500733520329 | 27.450073356685994 | -0.000000004653094 |
| 4 | 11.8248343539311 | 11.824834303775908 | 0.000000050155192 |
| 5 | 14.6350201943092 | 14.635020084480068 | 0.000000109829132 |
| 6 | 9.47801965023193 | 9.47801959933821 | 0.00000005089372 |
| 7 | 13.7277971543547 | 13.727797185025869 | -0.000000030671169 |
| 8 | 12.1724361254889 | 12.172436124250249 | 0.000000001238651 |
| 9 | 17.8483094625563 | 17.848309462917722 | -0.000000000361522 |
| 10 | 11.4141953861842 | 11.414195386232246 | -0.000000000047954 |
| 11 | 16.1900735681208 | 16.1900735700657 | -0.000000001944900 |
| 12 | 12.385319820021 | 12.385319820401966 | -0.000000000380966 |
| 13 | 10.4332040960123 | 10.43320407661182 | 0.00000001940048 |
| 14 | 41.7785075156215 | 41.7785074506677 | 0.0000000649538 |
| 15 | 38.0121380265941 | 38.01213778080117 | 0.00000024579293 |
| 16 | 27.1439462145094 | 27.14394624582416 | -0.00000003131476 |
| 17 | 9.50105793733591 | 9.501057920471283 | 0.000000016864627 |
| 18 | 18.9383195599305 | 18.93831947016505 | 0.00000008976545 |
| 19 | 17.3566153448881 | 17.35661536394406 | -0.00000001905596 |
| 20 | 19.450701347322 | 19.450701334660998 | 0.000000012661002 |
| 21 | 15.1800288763207 | 15.180028847571453 | 0.000000028749247 |
| 22 | 17.5135203624483 | 17.513520315329163 | 0.000000047119137 |
| 23 | 15.1051590271628 | 15.105158902227114 | 0.000000124935686 |
| 24 | 35.5388334703914 | 35.538833666779524 | -0.000000196388124 |
| 25 | 21.3261982841632 | 21.32619837250057 | -0.00000008833737 |
| 26 | 9.027353231579 | 9.027353232868238 | -0.000000001289238 |
| 27 | 30.1307246819617 | 30.130724681830557 | 0.000000000131143 |
| 28 | 19.6400677244916 | 19.640067727176127 | -0.000000002684527 |
| 29 | 12.2678522027652 | 12.267852189968524 | 0.000000012796676 |
| 30 | 24.6410930688427 | 24.641092604213856 | 0.000000464628844 |

**結論**：R 與 Python 的結果高度一致，差異範圍在 10^-7 到 10^-10 之間，證明 Python 實現的正確性。

### Plotting functions

In [54]:
# for i in range(test_data.shape[1]):
#     y_pred = pred['pred.value'][:, i].detach().cpu()
#     y_true = test_data[:, i].detach().cpu()

#     tmp = F.mse_loss(y_pred, y_true)
#     print(tmp.item())

#     plt.figure(figsize=(8,5))
#     plt.plot(y_true, label='True Value', marker='o')
#     plt.plot(y_pred, label='Predicted Value', marker='x')
#     plt.xlabel('Sample Index')
#     plt.ylabel('Value')
#     plt.title(f'Prediction vs True Value at {i} Step with error {round(tmp.item(), 2)}')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

### Check gradient tracking

In [53]:
# from torchviz import make_dot
# SAVE_DIR = "gradient tracking/with unknown locations"
# os.makedirs(SAVE_DIR, exist_ok=True)

# for k, v in model.obj.items():
#     if isinstance(v, torch.Tensor):
#         print(f"{k}: requires_grad={v.requires_grad};  grad_fn={v.grad_fn}; is_leaf={v.is_leaf}")
#         # file_path = os.path.join(SAVE_DIR, k)
#         # dot = make_dot(v)
#         # dot.attr(dpi='300')
#         # dot.render(file_path, format='png')

# for k, v in model.obj['G'].items():
#     if isinstance(v, torch.Tensor):
#         print(f"{k}: requires_grad={v.requires_grad};  grad_fn={v.grad_fn}; is_leaf={v.is_leaf}")
#         # file_path = os.path.join(SAVE_DIR, k)
#         # dot = make_dot(v)
#         # dot.attr(dpi='300')
#         # dot.render(file_path, format='png')

# for k, v in pred.items():
#     if isinstance(v, torch.Tensor):
#         print(f"{k}: requires_grad={v.requires_grad};  grad_fn={v.grad_fn}; is_leaf={v.is_leaf}")
#         # file_path = os.path.join(SAVE_DIR, k)
#         # dot = make_dot(v)
#         # dot.attr(dpi='300')
#         # dot.render(file_path, format='png')

M: requires_grad=True;  grad_fn=<MmBackward0 object at 0x000001565B73F9A0>; is_leaf=False
s: requires_grad=True;  grad_fn=<SubBackward0 object at 0x000001565B77EEF0>; is_leaf=False
negloglik: requires_grad=True;  grad_fn=<AddBackward0 object at 0x000001565B77EEF0>; is_leaf=False
w: requires_grad=True;  grad_fn=<CopySlices object at 0x000001565B77EEF0>; is_leaf=False
V: requires_grad=True;  grad_fn=<SubBackward0 object at 0x000001565B77EEF0>; is_leaf=False
MRTS: requires_grad=False;  grad_fn=None; is_leaf=True
UZ: requires_grad=False;  grad_fn=None; is_leaf=True
Xu: requires_grad=False;  grad_fn=None; is_leaf=True
nconst: requires_grad=False;  grad_fn=None; is_leaf=True
BBBH: requires_grad=False;  grad_fn=None; is_leaf=True
pred.value: requires_grad=True;  grad_fn=<AddBackward0 object at 0x000001565B77EEF0>; is_leaf=False


## Test on missing data (EM)

### Fit and predict

In [94]:
# training on data with missing values
model = AutoFRK(
    logger_level=20
)
model.forward(
    data=data_missing,
    loc=locs,
    method='EM',
    requires_grad=True,
    maxit=18
)

# predict on data with missing values
pred = model.predict(
    obj = model.obj
)
pred

2025-10-24 00:27:28 - autoFRK.utils.logger - WARNING: Parameter "device" was not set. Value "cuda:0" detected and used.
2025-10-24 00:27:28 - autoFRK.utils.logger - INFO: Gradient tracking has been enabled for autoFRK.
2025-10-24 00:27:28 - autoFRK.utils.logger - INFO: Calculate TPS with rectangular coordinates.
2025-10-24 00:28:20 - autoFRK.utils.logger - INFO: Number of iteration: 18


{'pred.value': tensor([[-3.9923e-04, -3.9498e+00,  4.0037e+00,  ..., -1.4526e+00,
          -3.7519e-01,  4.1146e+00],
         [-2.9209e-01, -3.0439e+00,  2.9015e+00,  ..., -9.5498e-01,
           2.6059e-01,  2.9861e+00],
         [-6.3041e+00, -8.5653e+00,  7.0040e-01,  ...,  3.5755e-01,
          -2.5926e+00,  1.3498e+01],
         ...,
         [ 1.6149e+00, -1.1902e+00,  3.4781e+00,  ..., -1.4641e+00,
           6.4208e-01, -2.6462e-01],
         [-7.2337e+00, -4.5718e+00, -5.4361e+00,  ...,  1.9453e+00,
          -3.4074e+00,  9.6046e+00],
         [-8.9564e-01, -7.6079e-01, -1.4871e-02,  ...,  2.4362e-01,
           5.3259e-01,  8.8942e-01]], device='cuda:0', dtype=torch.float64,
        grad_fn=<AddBackward0>),
 'se': None}

### Compute MSE

In [101]:
F.mse_loss(pred['pred.value'].cpu(), data.cpu()).item()

5.115453562007604

In [105]:

# Initialize dictionary to store MSE for each time step
mse_per_time_step = {}

# Each time step MSE
for i in range(data.shape[1]):
    y_pred = pred['pred.value'][:, i].cpu()
    y_true = data[:, i].cpu()

    tmp = F.mse_loss(y_pred, y_true)
    # Store the MSE for each time step
    mse_per_time_step[i] = tmp.item()
    print(f"Time step {i} MSE: {tmp.item()}")

Time step 0 MSE: 4.8759131017370265
Time step 1 MSE: 4.369524709645461
Time step 2 MSE: 4.602030801880705
Time step 3 MSE: 6.587195417892187
Time step 4 MSE: 5.064184366646276
Time step 5 MSE: 5.202673232390709
Time step 6 MSE: 5.372427594280316
Time step 7 MSE: 5.224291304748898
Time step 8 MSE: 5.398161469882073
Time step 9 MSE: 4.887758970521392
Time step 10 MSE: 5.3842491546734825
Time step 11 MSE: 4.882234535267565
Time step 12 MSE: 3.4528505253076234
Time step 13 MSE: 5.230932513008856
Time step 14 MSE: 5.776500950613535
Time step 15 MSE: 5.1688759087939555
Time step 16 MSE: 5.576635410948221
Time step 17 MSE: 5.836962497348546
Time step 18 MSE: 5.143071764861445
Time step 19 MSE: 5.82781098321492
Time step 20 MSE: 5.915852331902606
Time step 21 MSE: 4.021226662025229
Time step 22 MSE: 4.681773734924411
Time step 23 MSE: 5.727550865857844
Time step 24 MSE: 3.408651673041818
Time step 25 MSE: 4.571510938322594
Time step 26 MSE: 4.726495724858257
Time step 27 MSE: 5.616360174014176

{0: 4.8759131017370265,
 1: 4.369524709645461,
 2: 4.602030801880705,
 3: 6.587195417892187,
 4: 5.064184366646276,
 5: 5.202673232390709,
 6: 5.372427594280316,
 7: 5.224291304748898,
 8: 5.398161469882073,
 9: 4.887758970521392,
 10: 5.3842491546734825,
 11: 4.882234535267565,
 12: 3.4528505253076234,
 13: 5.230932513008856,
 14: 5.776500950613535,
 15: 5.1688759087939555,
 16: 5.576635410948221,
 17: 5.836962497348546,
 18: 5.143071764861445,
 19: 5.82781098321492,
 20: 5.915852331902606,
 21: 4.021226662025229,
 22: 4.681773734924411,
 23: 5.727550865857844,
 24: 3.408651673041818,
 25: 4.571510938322594,
 26: 4.726495724858257,
 27: 5.616360174014176,
 28: 5.537758206815772,
 29: 5.392141334802216}

### Compare with R result

### Plotting functions